In [1]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

Ahora procedemos a cargar el fichero

In [2]:
file_path = "input_wbs.ods"
data = get_data(file_path)['Export']
tasks = {}
print("file loaded")

file loaded


In [3]:
cfg_prefix = "wp"
cfg_name_prefix = "WP"
cfg_tgt_prefix = "T"
cfg_first_valid_row = 2
cfg_import_column = 0
cfg_level_column = 3
cfg_ident_column = 1
cfg_name_column = 16
cfg_parent_column = 13
cfg_description_column = 17

cfg_depend_column = 4
cfg_duration_column = 9
cfg_assignee_column = 11
cfg_rmtracker_id = 3
cfg_project_prefix_cfid = 21
cfg_rmtitle_cfid = 16
rm_cfield_dep = 32


In [4]:
def processTgt(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    thisTgt = {}
    print("tgt:",row[cfg_ident_column])
    thisTgt['level'] = level
    thisTgt['description'] = row[cfg_name_column]
    if (len(row) > cfg_description_column):
        thisTgt['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        thisTgt['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['tgtnum'] + 1
        thisTgt['ident'] = projects[row[cfg_parent_column]]['name'] + "."+cfg_tgt_prefix+"{:02d}".format(idx)
        projects[row[cfg_parent_column]]['tgtnum'] = idx
        projects[row[cfg_parent_column]]['targets'][row[cfg_ident_column]] = thisTgt

    else:
        thisTgt['parent'] = None
        idx = projects['root']['tgtnum'] + 1
        thisTgt['ident'] = cfg_tgt_prefix + "{:02d}".format(idx)
        projects['root']['tgtnum'] = idx
        projects['root']['targets'][row[cfg_ident_column]] = thisTgt
    
    return thisTgt    

def processPrj(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    thisPrj = {}
    print("prj:",row[1])
    thisPrj['level'] = level
    thisPrj['description'] = row[cfg_name_column]
    thisPrj['childnum'] = 0
    thisPrj['tgtnum'] = 0
    thisPrj['targets'] = {}
    if (len(row) > cfg_description_column):
        thisPrj['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        thisPrj['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['childnum'] + 1
        projects[row[cfg_parent_column]]['childnum'] = idx
        thisPrj['name'] = projects[row[cfg_parent_column]]['name'] + ".{:02d}".format(idx)
        thisPrj['ident'] = projects[row[cfg_parent_column]]['ident'] + "-{:02d}".format(idx)

    else:
        thisPrj['parent'] = None
        idx = projects['root']['childnum'] + 1
        projects['root']['childnum'] = idx
        thisPrj['name'] = cfg_name_prefix + "{:02d}".format(idx)
        thisPrj['ident'] = cfg_prefix + "{:02d}".format(idx)
    
    return thisPrj

def process_row(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    if (level > 0):
        lastPrj = processPrj(row,lastPrj,projects)
        projects[row[cfg_ident_column]] = lastPrj
    else:
        lastTgt = processTgt(row, lastPrj, projects)
        targets[row[cfg_ident_column]] = lastTgt
        
    return lastPrj

i = 0
projects = {}
projects['root'] = {}
projects['root']['childnum'] = 0
projects['root']['tgtnum'] = 0
projects['root']['targets'] = {}
lastPrj = None
targets = {}

for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        if (len(row) > cfg_import_column):
            if (row[cfg_import_column]>0):
                print("processing row",i)
                lastPrj = process_row(row,lastPrj,projects)

    i += 1
    
print("done")

processing row 2
prj: A1
processing row 3
prj: A2
processing row 4
tgt: A3
processing row 5
tgt: A4
processing row 6
tgt: A5
processing row 7
tgt: A6
processing row 8
prj: A7
processing row 9
tgt: A8
processing row 10
tgt: A9
processing row 11
prj: A10
processing row 12
prj: A11
processing row 13
prj: A12
processing row 14
prj: A13
processing row 15
prj: A14
processing row 16
prj: A15
processing row 17
prj: A16
processing row 18
prj: A17
processing row 19
prj: A18
processing row 20
tgt: A19
processing row 21
tgt: A20
processing row 22
prj: A21
processing row 23
prj: A22
processing row 24
tgt: A23
processing row 25
tgt: A24
processing row 26
prj: A25
processing row 27
prj: A26
processing row 28
prj: A27
processing row 29
prj: A28
processing row 30
prj: A29
processing row 31
tgt: A30
processing row 32
prj: A31
processing row 33
prj: A32
processing row 34
tgt: A33
processing row 35
tgt: A34
processing row 36
prj: A35
processing row 37
tgt: A36
processing row 38
prj: A37
processing row 39


In [5]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))
            
pretty(projects)

root
	childnum
		16
	tgtnum
		0
	targets
A1
	level
		1
	description
		EST Project Management 
	childnum
		9
	tgtnum
		6
	targets
		A3
			level
				0
			description
				Management Plan 

Este plan es el resumen de todo el paquete de trabajo.
			parent
				A1
			ident
				WP01.T01
		A4
			level
				0
			description
				Construction Proposal 
			parent
				A1
			ident
				WP01.T02
		A5
			level
				0
			description
				PO proposal for Construction phase

Es la propuesta de personal, organigrama y coste para la fase de construcción
			parent
				A1
			ident
				WP01.T03
		A6
			level
				0
			description
				Preparatory Phase Funding Justification 
			parent
				A1
			ident
				WP01.T04
		A8
			level
				0
			description
				Safety Assurance 

Verificación de cumplimiento de las normas de seguridad
			parent
				A1
			ident
				WP01.T05
		A9
			level
				0
			description
				Product and Quality Assurance 

Garantía de consecusión de seguridad y calidad del producto.
			parent
				A1
			ide

In [6]:
pretty(targets)

A3
	level
		0
	description
		Management Plan 

Este plan es el resumen de todo el paquete de trabajo.
	parent
		A1
	ident
		WP01.T01
A4
	level
		0
	description
		Construction Proposal 
	parent
		A1
	ident
		WP01.T02
A5
	level
		0
	description
		PO proposal for Construction phase

Es la propuesta de personal, organigrama y coste para la fase de construcción
	parent
		A1
	ident
		WP01.T03
A6
	level
		0
	description
		Preparatory Phase Funding Justification 
	parent
		A1
	ident
		WP01.T04
A8
	level
		0
	description
		Safety Assurance 

Verificación de cumplimiento de las normas de seguridad
	parent
		A1
	ident
		WP01.T05
A9
	level
		0
	description
		Product and Quality Assurance 

Garantía de consecusión de seguridad y calidad del producto.
	parent
		A1
	ident
		WP01.T06
A19
	level
		0
	description
		Science Requirements Document  

Definición de los requerimientos científicos para EST (Science Cases)
	parent
		A17
	ident
		WP02.T01
A20
	level
		0
	description
		Observing Cases 

Definici

In [7]:
print(projects.keys())

dict_keys(['root', 'A1', 'A2', 'A7', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A25', 'A26', 'A27', 'A28', 'A29', 'A31', 'A32', 'A35', 'A37', 'A38', 'A39', 'A40', 'A41', 'A47', 'A48', 'A49', 'A50', 'A51', 'A54', 'A55', 'A56', 'A59', 'A61', 'A64', 'A74', 'A75', 'A76', 'A77', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 'A93', 'A94', 'A97', 'A98', 'A99', 'A105', 'A106', 'A109', 'A111', 'A114', 'A124', 'A125', 'A128', 'A130', 'A133', 'A143', 'A144', 'A147', 'A149', 'A152', 'A154', 'A155', 'A156', 'A157', 'A160', 'A162', 'A165', 'A168', 'A169', 'A170', 'A171', 'A174', 'A176', 'A179', 'A182', 'A183', 'A184', 'A185', 'A188', 'A192', 'A193', 'A197', 'A204', 'A205', 'A206', 'A207', 'A208', 'A209', 'A210', 'A211', 'A212', 'A213', 'A214', 'A229', 'A235', 'A242', 'A247', 'A248', 'A251', 'A252', 'A253', 'A254', 'A256', 'A257', 'A258', 'A259', 'A261', 'A262', 'A263', 'A264', 'A265', 'A266', 'A267', 'A268', 'A269', 'A270', 'A271', 'A272', 'A275', 'A276', 'A277'

In [8]:
print(targets.keys())

dict_keys(['A3', 'A4', 'A5', 'A6', 'A8', 'A9', 'A19', 'A20', 'A23', 'A24', 'A30', 'A33', 'A34', 'A36', 'A42', 'A43', 'A44', 'A45', 'A46', 'A52', 'A53', 'A57', 'A58', 'A60', 'A62', 'A63', 'A65', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A78', 'A79', 'A80', 'A81', 'A82', 'A83', 'A91', 'A92', 'A95', 'A96', 'A100', 'A101', 'A102', 'A103', 'A104', 'A107', 'A108', 'A110', 'A112', 'A113', 'A115', 'A116', 'A117', 'A118', 'A119', 'A120', 'A121', 'A122', 'A123', 'A126', 'A127', 'A129', 'A131', 'A132', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A140', 'A141', 'A142', 'A145', 'A146', 'A148', 'A150', 'A151', 'A153', 'A158', 'A159', 'A161', 'A163', 'A164', 'A166', 'A167', 'A172', 'A173', 'A175', 'A177', 'A178', 'A180', 'A181', 'A186', 'A187', 'A189', 'A190', 'A191', 'A194', 'A195', 'A196', 'A198', 'A199', 'A200', 'A201', 'A202', 'A203', 'A215', 'A216', 'A217', 'A218', 'A219', 'A220', 'A221', 'A222', 'A223', 'A224', 'A225', 'A226', 'A227', 'A228', 'A230', 'A231', 'A232', 'A233', 

Vamos a ver si ya existen los proyectos, y los vamos a anidar en la estructura que ya tenemos, o los crearemos nuevos.

In [10]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
rmprojects = redmine.project.all()

rmprojectsbyident = {}
print("Proyectos:")
for p in rmprojects:
    print ("\t",p.identifier," \t| ",p.name)
    rmprojectsbyident[p.identifier] = p

root_rmproject = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto raíz: ",root_rmproject.identifier," | ",root_rmproject.name)

Proyectos:
	 est  	|  EST
	 wp01  	|  WP01
	 wp01-01  	|  WP01.01
	 wp01-02  	|  WP01.02
	 wp01-03  	|  WP01.03
	 wp01-04  	|  WP01.04
	 wp01-05  	|  WP01.05
	 wp01-06  	|  WP01.06
	 wp01-07  	|  WP01.07
	 wp01-08  	|  WP01.08
	 wp01-09  	|  WP01.09
	 wp02  	|  WP02
	 wp02-01  	|  WP02.01
	 wp02-02  	|  WP02.02
	 wp02-02-01  	|  WP02.02.01
	 wp02-03  	|  WP02.03
	 wp02-04  	|  WP02.04
	 wp03  	|  WP03
	 wp03-01  	|  WP03.01
	 wp03-02  	|  WP03.02
	 wp03-02-01  	|  WP03.02.01
	 wp03-03  	|  WP03.03
	 wp03-04  	|  WP03.04
	 wp03-04-01  	|  WP03.04.01
	 wp03-05  	|  WP03.05
	 wp03-06  	|  WP03.06
	 wp03-07  	|  WP03.07
	 wp03-08  	|  WP03.08
	 wp03-08-01  	|  WP03.08.01
	 wp03-08-02  	|  WP03.08.02
	 wp03-09  	|  WP03.09
	 wp03-10  	|  WP03.10
	 wp03-11  	|  WP03.11
	 wp03-12  	|  WP03.12
	 wp04  	|  WP04
	 wp04-01  	|  WP04.01
	 wp04-02  	|  WP04.02
	 wp04-02-01  	|  WP04.02.01
	 wp04-02-02  	|  WP04.02.02
	 wp05  	|  WP05
	 wp05-01  	|  WP05.01
	 wp05-01-01  	|  WP05.01.01
	 wp05-01-02 

In [11]:
for k in projects.keys():
    if (k != 'root'):
        thisproject = projects[k]
        prident = thisproject['ident']
        print(k,prident)
        print(thisproject)
        rmprj = None
        if (prident in rmprojectsbyident.keys()):
            print("Está en la lista, lo busco")
            rmprj = redmine.project.get(prident)
            if (rmprj is not None):
                print("Lo hemos encontrado")
                projects[k]['rmprj'] = rmprj
                
        if (thisproject['parent'] is None):
            print("No hay padre")
            rmprojectid = root_rmproject.id
            
        else:
            print("Sí hay padre")
            rmprojectid = projects[thisproject['parent']]['rmid']
        
        print("rmprojectid",rmprojectid,thisproject['ident'])
          
        if rmprj is None:
            rmprj = redmine.project.create(
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                    parent_id=rmprojectid,
                    inherit_members=True
            )            
        else:
            redmine.project.update(
                    rmprj.id,
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                
                    parent_id=rmprojectid,
                
                    inherit_members=True
            )
        
        thisproject['rmid'] = rmprj.id                

        for v in rmprj.versions:
            redmine.version.delete(v.id)
        
        for k in thisproject['targets']:
            thistgt = targets[k]
            tgtident = thistgt['ident']
            print(k,tgtident)
            print(thistgt)
        
            rmtgt = None
            for v in rmprj.versions:
                print("version:",v.name,"?",thistgt['ident'])
                if v.name == thistgt['ident']:
                    print("lo encontré")
                    rmtgt = v

            if rmtgt is None:
                print("Creamos el target")
                rmtgt = redmine.version.create(
                     project_id=rmprj.id,
                     name=thistgt['ident'],
                     description=thistgt['description'][:250]
                )            

            else:
                print("Modificamos el target")
                redmine.version.update(
                    rmtgt.id,
                    name=thistgt['ident'],
                    description=thistgt['description'][:250]
                )            
 












A1 wp01
{'level': 1, 'description': 'EST Project Management ', 'childnum': 9, 'tgtnum': 6, 'targets': {'A3': {'level': 0, 'description': 'Management Plan \n\nEste plan es el resumen de todo el paquete de trabajo.', 'parent': 'A1', 'ident': 'WP01.T01'}, 'A4': {'level': 0, 'description': 'Construction Proposal ', 'parent': 'A1', 'ident': 'WP01.T02'}, 'A5': {'level': 0, 'description': 'PO proposal for Construction phase\n\nEs la propuesta de personal, organigrama y coste para la fase de construcción', 'parent': 'A1', 'ident': 'WP01.T03'}, 'A6': {'level': 0, 'description': 'Preparatory Phase Funding Justification ', 'parent': 'A1', 'ident': 'WP01.T04'}, 'A8': {'level': 0, 'description': 'Safety Assurance \n\nVerificación de cumplimiento de las normas de seguridad', 'parent': 'A1', 'ident': 'WP01.T05'}, 'A9': {'level': 0, 'description': 'Product and Quality Assurance \n\nGarantía de consecusión de seguridad y calidad del producto.', 'parent': 'A1', 'ident': 'WP01.T06'}}, 'parent': None, 'na

In [12]:
print("done")

done


Code for removing versions (disabled)

In [ ]:
'''rmprojects = redmine.project.all()

for k in rmprojects:
    print(k.name)
    for v in k.versions:
        print(v.name)
        redmine.version.delete(v.id)
'''
